In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#data visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#stats
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm

#expand output
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
fifa_train = pd.read_csv(r"C:\Users\ssara\OneDrive\Ambiente de Trabalho\Class 1\FIFA Project\FIFA_TRAIN_DATA.csv", sep= '?')
fifa_test = pd.read_csv(r"C:\Users\ssara\OneDrive\Ambiente de Trabalho\Class 1\FIFA Project\FIFA_TEST_DATA.csv", sep= '?')

In [3]:
#header cleaning_we have removed the space , capital letters to uniformize the selection

def transformation (df1):
    cols=[]
    for i in range(len(df1.columns)):
        cols.append(df1.columns[i].lower().replace(' ','_'))
    df1.columns=cols
    
transformation (fifa_test) 
transformation (fifa_train)

In [4]:
#We found the intercept between the 2 DF and we found the 3 columns below were not in common
#one of them 'ova' is the variable we want to predict - 'bov' & 'pot' we are going to save in a variable and then drop from train
'''
BOV" typically stands for "Ball Over the Top."-->we can drop this column
******OVA" likely refers to "Overall******=Y
POT" stands for "Potential -->second Y if you build a Function
'''
columns_not_comm = fifa_test.columns.symmetric_difference(fifa_train.columns)
columns_not_comm = pd.DataFrame(columns_not_comm)
columns_not_comm

,0
0,bov
1,ova
2,pot


DATA CLEANING ACTIVITY

FILLING & CATEGORIZING & ELIMINATING & CONVERTING COLUMNS

In [5]:
#we found the following columns containing useful statistics for the predictions contained null values - 
#we have used the mean method to fill and replace the columns for both the datasets

fifa_train['volleys'] = fifa_train['volleys'].fillna(fifa_train['volleys'].mean())
fifa_train['curve'] = fifa_train['curve'].fillna(fifa_train['curve'].mean())
fifa_train['agility'] = fifa_train['agility'].fillna(fifa_train['agility'].mean())
fifa_train['balance'] = fifa_train['balance'].fillna(fifa_train['balance'].mean())
fifa_train['jumping'] = fifa_train['jumping'].fillna(fifa_train['jumping'].mean())
fifa_train['vision'] = fifa_train['vision'].fillna(fifa_train['vision'].mean())
fifa_train['interceptions'] = fifa_train['interceptions'].fillna(fifa_train['interceptions'].mean())
fifa_train['positioning'] = fifa_train['positioning'].fillna(fifa_train['positioning'].mean())
fifa_train['composure'] = fifa_train['composure'].fillna(fifa_train['composure'].mean())
fifa_train['sliding_tackle'] = fifa_train['sliding_tackle'].fillna(fifa_train['sliding_tackle'].mean())

fifa_test['volleys'] = fifa_test['volleys'].fillna(fifa_test['volleys'].mean())
fifa_test['curve'] = fifa_test['curve'].fillna(fifa_test['curve'].mean())
fifa_test['agility'] = fifa_test['agility'].fillna(fifa_test['agility'].mean())
fifa_test['balance'] = fifa_test['balance'].fillna(fifa_test['balance'].mean())
fifa_test['jumping'] = fifa_test['jumping'].fillna(fifa_test['jumping'].mean())
fifa_test['vision'] = fifa_test['vision'].fillna(fifa_test['vision'].mean())
fifa_test['composure'] = fifa_test['composure'].fillna(fifa_test['composure'].mean())
fifa_test['sliding_tackle'] = fifa_test['sliding_tackle'].fillna(fifa_test['sliding_tackle'].mean())

In [6]:
# our goal is to predict the y by categorizing our dataset into 4 main bins: goalk., defense, midfie. and attack
#we performd a np.where to go through the selected column containing the position 'bp' and creating a new reduced one by our 4 categories

fifa_train['pos_categ'] =  np.where(fifa_train['bp'] == 'CB', 'def',np.where(fifa_train['bp'] == 'RWB', 'def', 
                         np.where(fifa_train['bp'] == 'LB','def',
                         np.where(fifa_train['bp'] == 'LWB', 'def',np.where(fifa_train['bp'] == 'RB', 'def',
                          np.where( fifa_train['bp'] == 'CM', 'mid',np.where(fifa_train['bp'] == 'CAM', 'mid',
                        np.where(fifa_train['bp'] == 'CDM', 'mid',
                          np.where( fifa_train['bp'] == 'RM', 'mid',np.where(fifa_train['bp'] == 'LM', 'mid',
                           np.where(fifa_train['bp'] == 'CF', 'att', np.where(fifa_train['bp'] == 'ST', 'att',
                             np.where(fifa_train['bp'] == 'RW', 'att', np.where( fifa_train['bp'] == 'LW', 'att','gkp'))))))))))))))

In [7]:
fifa_test['pos_categ'] =  np.where(fifa_test['bp'] == 'CB', 'def',np.where(fifa_test['bp'] == 'RWB', 'def', 
                             np.where(fifa_test['bp'] == 'LB','def',
                             np.where(fifa_test['bp'] == 'LWB', 'def',np.where(fifa_test['bp'] == 'RB', 'def',
                              np.where( fifa_test['bp'] == 'CM', 'mid',np.where(fifa_test['bp'] == 'CAM', 'mid',
                            np.where(fifa_test['bp'] == 'CDM', 'mid',
                              np.where( fifa_test['bp'] == 'RM', 'mid',np.where(fifa_test['bp'] == 'LM', 'mid',
                               np.where(fifa_test['bp'] == 'CF', 'att', np.where(fifa_test['bp'] == 'ST', 'att',
                                 np.where(fifa_test['bp'] == 'RW', 'att', np.where( fifa_test['bp'] == 'LW', 'att','gkp'))))))))))))))

In [8]:
#we dropped other not usefull or categorical columns we have found in our research

column_to_drop= ['unnamed:_0', 'nationality', 'club', 'bp', 'position',
                 'player_photo', 'club_logo', 'flag_photo', 'team_&_contract', 'foot', 
                 'joined', 'release_clause','loan_date_end', 'contract','gender','a/w','d/w']
fifa_train = fifa_train.drop(column_to_drop, axis =1)
fifa_test = fifa_test.drop(column_to_drop, axis =1)

In [9]:
#using the below funtion we clean the column value and wage by extracting the €/K/M and converting into Milion or Thousend + integer

def monetary_conversion (Value):
    if isinstance(Value,str): #evaluate if the statment is true or false based on the second arguement
        drop_ccy_amount = Value.replace('€','')
        if 'M' in drop_ccy_amount:
            drop_ccy_amount =float(drop_ccy_amount.replace('M',''))*1000000
        elif 'K' in Value:
            drop_ccy_amount=float(drop_ccy_amount.replace('K',''))*1000
        return float(drop_ccy_amount)
           
fifa_train['value'] = fifa_train['value'].apply(lambda x: monetary_conversion(x)) #we apply the def using lamba to the column selected
fifa_train['wage'] = fifa_train['wage'].apply(lambda x: monetary_conversion(x))

fifa_test['value'] = fifa_test['value'].apply(lambda x: monetary_conversion(x))
fifa_test['wage'] = fifa_test['wage'].apply(lambda x: monetary_conversion(x))

In [10]:
#the goal of the below is to clean the height by extraction the values inside, feet and inches and multiplying by the respective correspondent value in unit Metric

feet_inches = fifa_train['height'].str.split("'", expand=True)
feet_inches1 = fifa_test['height'].str.split("'", expand=True)

feet = feet_inches[0].astype(int)
feet1 = feet_inches1[0].astype(int)

inches = feet_inches[1].str.strip('"').astype(int)
inches1 = feet_inches1[1].str.strip('"').astype(int)

total_inches = 100 +feet * 12 + inches 
total_inches1 = 100 +feet1 * 12 + inches1

fifa_train['height'] = total_inches
fifa_test['height'] = total_inches1

In [11]:
#the goal of the below is to clean the weight by extraction the values inside lbr and multiplying by the respective correspondent value in unit kilograms 


lbs= fifa_train['weight'].str.split("lbs", expand=True)
lbs1= fifa_test['weight'].str.split("lbs", expand=True)

lbs = lbs[0].astype(int)
lbs1 = lbs1[0].astype(int)

fifa_train['weight'] = round(lbs/2.205,0)
fifa_test['weight'] = round(lbs1/2.205,0)


In [12]:
#we figured out the problem with the columns with math. operation inside - the below evaluate every cells in the column and performed the sum or the diff

'''
-eval() to dynamically evaluate expressions from a string-based or compiled-code-based input.
-The try block lets you test a block of code for errors.
-The except block lets you handle the error.
-The applymap() function is used to all cells of the Dataframe.
'''

def evaluate_expression(expression):
    try:
        result = eval(expression)
        return result
    except:
        return expression

fifa_train = fifa_train.applymap(evaluate_expression)
fifa_test = fifa_test.applymap(evaluate_expression)

In [13]:
# Cleaning the stars from the columns SM, W/F, ir and replaicing with a numerical value to be used 


fifa_train['sm'] = fifa_train['sm'].str.extract(r'(\d+)', expand=False)
fifa_train['w/f'] = fifa_train['w/f'].str.extract(r'(\d+)', expand=False)
fifa_train['ir'] = fifa_train['ir'].str.extract(r'(\d+)', expand=False)

fifa_test['sm'] = fifa_test['sm'].str.extract(r'(\d+)', expand=False)
fifa_test['w/f'] = fifa_test['w/f'].str.extract(r'(\d+)', expand=False)
fifa_test['ir'] = fifa_test['ir'].str.extract(r'(\d+)', expand=False)

fifa_train['sm'] = pd.to_numeric(fifa_train['sm'], errors='coerce')
fifa_train['w/f'] = pd.to_numeric(fifa_train['w/f'], errors='coerce')
fifa_train['ir'] = pd.to_numeric(fifa_train['ir'], errors='coerce')
fifa_test['sm'] = pd.to_numeric(fifa_test['sm'], errors='coerce')
fifa_test['w/f'] = pd.to_numeric(fifa_test['w/f'], errors='coerce')
fifa_test['ir'] = pd.to_numeric(fifa_test['ir'], errors='coerce')
fifa_train['hits'] = pd.to_numeric(fifa_train['hits'], errors='coerce')
fifa_test['hits'] = pd.to_numeric(fifa_test['hits'], errors='coerce')

Creating 4 groups - gk; def; mid; att;

In [14]:
#based on the category we created we split our datasets in 4 sub-datasets to performed separated regression

In [15]:
cond= fifa_train['pos_categ']=='gkp'
cond_1= fifa_test['pos_categ']=='gkp'

gk_fifa_train = fifa_train[cond]
gk_fifa_test = fifa_test[cond_1]

In [16]:
cond1= fifa_train['pos_categ']=='def'
cond1_1= fifa_test['pos_categ']=='def'

def_fifa_train = fifa_train[cond1]
def_fifa_test = fifa_test[cond1_1]

In [17]:
cond2= fifa_train['pos_categ']=='mid'
cond2_1= fifa_test['pos_categ']=='mid'

mid_fifa_train = fifa_train[cond2]
mid_fifa_test = fifa_test[cond2_1]

In [18]:
cond3= fifa_train['pos_categ']=='att'
cond3_1= fifa_test['pos_categ']=='att'

att_fifa_train = fifa_train[cond3]
att_fifa_test = fifa_test[cond3_1]

# Goalkeeper dataset 

In [19]:
gk_fifa_train_final = gk_fifa_train.drop(['ova','bov','id','pot','name','pos_categ','sm'], axis=1)

In [20]:
#this funtion aims to pass through our dataframe and clean correlation we believe could have multiculinearity
#the ova is not been considered in this funtion
#we consider relevant only up to 0.9 correlation
xcols = []
ycols = []
correlation = []
max_abs_correlation = 0.9 #our goal of correlation
for x in range(len(gk_fifa_train_final.columns)):
    for y in range(len(gk_fifa_train_final.corr())):
        if x != y:
            xcol = gk_fifa_train_final.columns[x] #new dataset with column and correlation
            ycol = gk_fifa_train_final.columns[y]
            if xcol in gk_fifa_train_final.columns and ycol in gk_fifa_train_final.columns:
                r = gk_fifa_train_final[xcol].corr(gk_fifa_train_final[ycol])
                correlation.append(r)
                xcols.append(xcol)
                ycols.append(ycol)
dropped_cols = set() #this remove duplicate values after iterattion w/ for loop
counter = 0
for i in range(len(correlation)): #we compare the new 2 datasets 
    if correlation[i] > abs(max_abs_correlation):
        xcol = xcols[i]
        ycol = ycols[i]
        if xcol in gk_fifa_train_final.columns and ycol in gk_fifa_train_final.columns:
            counter += 1
            # drop one of the columns with high correlation
            if xcol not in dropped_cols and ycol not in dropped_cols:
                if gk_fifa_train_final[xcol].std() > gk_fifa_train_final[ycol].std():
                    gk_fifa_train_final = gk_fifa_train_final.drop([ycol], axis=1)
                    dropped_cols.add(ycol)
                else:
                    gk_fifa_train_final = gk_fifa_train_final.drop([xcol], axis=1)
                    dropped_cols.add(xcol)
    else:
        pass

In [21]:
#merging my orginal df with my sorted for multicolinerarity issue --> rescuing my OVA

gk_fifa_train_final = pd.concat([gk_fifa_train_final, gk_fifa_train['ova']], axis=1)
gk_fifa_train_final.columns

Index(['age', 'height', 'weight', 'growth', 'value', 'wage', 'attacking', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling', 'curve', 'fk_accuracy', 'long_passing', 'ball_control', 'movement', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'mentality', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'composure', 'defending', 'marking', 'standing_tackle', 'sliding_tackle', 'total_stats', 'base_stats', 'w/f', 'ir', 'pas', 'hits', 'ram', 'rcb', 'ova'], dtype='object')

In [22]:
#after having tested with correlation, we decided to drop the below listed columns for the regression

drop_col_gk = ['height','growth','curve','fk_accuracy','long_passing','strength','long_shots',
             'composure','crossing','acceleration','sprint_speed','finishing']
gk_fifa_train_final = gk_fifa_train_final.drop(drop_col_gk, axis=1)

In [23]:
X_gk = gk_fifa_train_final[['age', 'weight', 'value', 'wage', 'attacking', 
                             'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling', 
                             'ball_control', 'movement', 'agility', 
                             'reactions', 'balance', 'power', 'shot_power', 'jumping', 'stamina', 
                             'mentality', 'aggression', 'interceptions', 'positioning', 'vision', 
                             'penalties', 'defending', 'marking', 'standing_tackle', 'sliding_tackle', 
                             'total_stats', 'base_stats', 'w/f', 'ir', 'pas', 'hits', 'ram', 'rcb']]
Y_gk = gk_fifa_train_final['ova']

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X_gk , Y_gk , test_size=0.2, random_state=42)

In [25]:
X_gk = sm.add_constant(X_gk)
model = sm.OLS(Y_gk,X_gk).fit()

In [26]:
lm_gk = linear_model.LinearRegression()
model = lm_gk.fit(X_train,Y_train)

lm_gk.score(X_train,Y_train)

0.9983217923414061

In [27]:
predictions_test_gk = lm_gk.predict(X_test)
predictions_train_gk = lm_gk.predict(X_train)

print("R2 value is = ",round(r2_score(Y_train, predictions_train_gk),2))

mse_gk = mean_squared_error(Y_test, predictions_test_gk)
mae_gk = mean_absolute_error(Y_test, predictions_test_gk)
print("The mse of the model is = ",mse_gk)
print("The root mse of the model is = ",np.sqrt(mse_gk))
print("The mean absolute error of the model is = ",mae_gk)

R2 value is =  1.0
The mse of the model is =  0.21539986199586944
The root mse of the model is =  0.46411190675942526
The mean absolute error of the model is =  0.3034754756848012


In [28]:
#plotting into a dataset my result
Y_predicted_gk = pd.concat([Y_test.reset_index(), pd.DataFrame(predictions_test_gk)], axis=1)
Y_predicted_gk.rename(columns = {0:'test_gk'}, inplace = True)
analysis_gk = Y_predicted_gk.sort_values(by = 'ova', ascending = False)
analysis_gk['Diff'] = (analysis_gk['ova'] - analysis_gk['test_gk']).round()
analysis_gk.sort_values(by = 'ova', ascending = False)
analysis_gk['test_gk']=analysis_gk['test_gk'].round()
analysis_gk

,index,ova,test_gk,Diff
46,2254,90,90.0,0.0
228,4224,88,88.0,0.0
75,2326,86,86.0,-0.0
26,7245,85,86.0,-1.0
31,6544,84,85.0,-1.0
37,5481,83,82.0,1.0
123,2146,83,84.0,-1.0
212,6050,82,83.0,-1.0
175,8424,82,81.0,1.0
73,241,82,82.0,0.0


In [29]:
#using the test to predict the result Y ova for the goal
X_gk_test = gk_fifa_test[['age', 'weight', 'value', 'wage', 'attacking', 
                             'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling', 
                             'ball_control', 'movement', 'agility', 
                             'reactions', 'balance', 'power', 'shot_power', 'jumping', 'stamina', 
                             'mentality', 'aggression', 'interceptions', 'positioning', 'vision', 
                             'penalties', 'defending', 'marking', 'standing_tackle', 'sliding_tackle', 
                             'total_stats', 'base_stats', 'w/f', 'ir', 'pas', 'hits', 'ram', 'rcb']]

predictions_test_gk = lm_gk.predict(X_gk_test)
gk_fifa_test['predictions']=predictions_test_gk.round()
final_gk = gk_fifa_test[['name','value', 'predictions']].sort_values(by='predictions', ascending = False)

# Defense start

In [30]:
def_fifa_train_final = def_fifa_train.drop(['ova','bov','id','pot','name','pos_categ','sm'], axis=1)

In [31]:
def_fifa_train_final['hits'] = def_fifa_train_final['hits'].fillna(def_fifa_train_final['hits'].mean())

In [32]:
xcols = []
ycols = []
correlation = []
max_abs_correlation = 0.9
for x in range(len(def_fifa_train_final.columns)):
    for y in range(len(def_fifa_train_final.corr())):
        if x != y:
            xcol = def_fifa_train_final.columns[x]
            ycol = def_fifa_train_final.columns[y]
            if xcol in def_fifa_train_final.columns and ycol in def_fifa_train_final.columns:
                r = def_fifa_train_final[xcol].corr(def_fifa_train_final[ycol])
                correlation.append(r)
                xcols.append(xcol)
                ycols.append(ycol)
dropped_cols = set()
counter = 0
for i in range(len(correlation)):
    if correlation[i] > abs(max_abs_correlation):
        xcol = xcols[i]
        ycol = ycols[i]
        if xcol in def_fifa_train_final.columns and ycol in def_fifa_train_final.columns:
            counter += 1
            # drop one of the columns with high correlation
            if xcol not in dropped_cols and ycol not in dropped_cols:
                if def_fifa_train_final[xcol].std() > def_fifa_train_final[ycol].std():
                    def_fifa_train_final = def_fifa_train_final.drop([ycol], axis=1)
                    dropped_cols.add(ycol)
                else:
                    def_fifa_train_final = def_fifa_train_final.drop([xcol], axis=1)
                    dropped_cols.add(xcol)
    else:
        pass

In [33]:
def_fifa_train_final = pd.concat([def_fifa_train_final, def_fifa_train['ova']], axis=1)

In [34]:
drop_col_def = ['volleys','dribbling','ball_control','movement','stamina','strength','aggression',
             'gk_diving','gk_handling','gk_positioning','gk_reflexes','total_stats','goalkeeping',
             'agility','wage','penalties','fk_accuracy','weight','hits','positioning','w/f']
def_fifa_train_final = def_fifa_train_final.drop(drop_col_def, axis=1)

In [35]:
X_def = def_fifa_train_final[['age', 'height', 'growth', 'value', 'crossing', 'finishing', 'heading_accuracy', 
                              'short_passing', 'curve', 'long_passing', 'sprint_speed', 'reactions', 'balance',
                              'power', 'shot_power', 'jumping', 'long_shots', 'interceptions', 'vision', 
                              'composure', 'defending', 'gk_kicking', 'ir', 'phy', 'rdm', 'gk']]
Y_def = def_fifa_train_final['ova']

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X_def , Y_def , test_size=0.2, random_state=42)

In [37]:
X_def = sm.add_constant(X_def)
model = sm.OLS(Y_def,X_def).fit()

In [38]:
lm_def = linear_model.LinearRegression()
model = lm_def.fit(X_train,Y_train)

lm_def.score(X_train,Y_train)

0.9653419087749711

In [39]:
predictions_test_def = lm_def.predict(X_test)
predictions_train_def = lm_def.predict(X_train)

print("R2 value is = ",round(r2_score(Y_train, predictions_train_def),2))
print("The intercept of the model is = ",lm_def.intercept_)

mse_def = mean_squared_error(Y_test, predictions_test_def)
mae_def = mean_absolute_error(Y_test, predictions_test_def)
print("The mse of the model is = ",mse_def)
print("The root mse of the model is = ",np.sqrt(mse_def))
print("The mean absolute error of the model is = ",mae_def)

R2 value is =  0.97
The intercept of the model is =  21.602122118051305
The mse of the model is =  1.670018500471133
The root mse of the model is =  1.292291956359372
The mean absolute error of the model is =  0.9929031197751782


In [40]:
Y_predicted_def = pd.concat([Y_test.reset_index(), pd.DataFrame(predictions_test_def)], axis=1)
Y_predicted_def.rename(columns = {0:'test_def'}, inplace = True)
analysis_def = Y_predicted_def.sort_values(by = 'ova', ascending = False)
analysis_def['Diff'] = (analysis_def['ova'] - analysis_def['test_def']).round()
analysis_def.sort_values(by = 'ova', ascending = False)
analysis_def['test_def']=analysis_def['test_def'].round()
analysis_def

,index,ova,test_def,Diff
171,544,89,90.0,-1.0
74,512,86,87.0,-1.0
791,1828,85,86.0,-1.0
431,8145,85,86.0,-1.0
378,736,85,85.0,-0.0
...,...,...,...,...
172,9952,49,54.0,-5.0
708,10597,49,49.0,-0.0
69,9598,49,50.0,-1.0
504,13339,49,51.0,-2.0


In [41]:
X_def_test = def_fifa_test[['age', 'height', 'growth', 'value', 'crossing', 'finishing', 'heading_accuracy', 
                              'short_passing', 'curve', 'long_passing', 'sprint_speed', 'reactions', 'balance',
                              'power', 'shot_power', 'jumping', 'long_shots', 'interceptions', 'vision', 
                              'composure', 'defending', 'gk_kicking', 'ir', 'phy', 'rdm', 'gk']]

predictions_test_def = lm_def.predict(X_def_test)
def_fifa_test['predictions']=predictions_test_def.round()
final_def = def_fifa_test[['name','value', 'predictions']].sort_values(by='predictions', ascending = False)

# Midfield start

In [42]:
mid_fifa_train_final = mid_fifa_train.drop(['ova','bov','id','pot','name','pos_categ','sm'], axis=1)

In [43]:
xcols = []
ycols = []
correlation = []
max_abs_correlation = 0.9
for x in range(len(mid_fifa_train_final.columns)):
    for y in range(len(mid_fifa_train_final.corr())):
        if x != y:
            xcol = mid_fifa_train_final.columns[x]
            ycol = mid_fifa_train_final.columns[y]
            if xcol in mid_fifa_train_final.columns and ycol in mid_fifa_train_final.columns:
                r = mid_fifa_train_final[xcol].corr(mid_fifa_train_final[ycol])
                correlation.append(r)
                xcols.append(xcol)
                ycols.append(ycol)
dropped_cols = set()
counter = 0
for i in range(len(correlation)):
    if correlation[i] > abs(max_abs_correlation):
        xcol = xcols[i]
        ycol = ycols[i]
        if xcol in mid_fifa_train_final.columns and ycol in mid_fifa_train_final.columns:
            counter += 1
            # drop one of the columns with high correlation
            if xcol not in dropped_cols and ycol not in dropped_cols:
                if mid_fifa_train_final[xcol].std() > mid_fifa_train_final[ycol].std():
                    mid_fifa_train_final = mid_fifa_train_final.drop([ycol], axis=1)
                    dropped_cols.add(ycol)
                else:
                    mid_fifa_train_final = mid_fifa_train_final.drop([xcol], axis=1)
                    dropped_cols.add(xcol)
    else:
        pass

In [44]:
mid_fifa_train_final = pd.concat([mid_fifa_train_final, mid_fifa_train['ova']], axis=1)

In [45]:
mid_fifa_train_final['hits'] = mid_fifa_train_final['hits'].fillna(mid_fifa_train_final['hits'].mean())

In [46]:
drop_col_def = ['w/f','long_shots','stamina','shot_power','acceleration', 'strength','dribbling','wage','weight','gk']
mid_fifa_train_final = mid_fifa_train_final.drop(drop_col_def, axis=1)

In [47]:
X_mid = mid_fifa_train_final[['age', 'height', 'growth', 'value', 'attacking', 'crossing', 'finishing', 
                            'heading_accuracy', 'volleys', 'skill', 'curve', 'fk_accuracy', 'long_passing', 
                            'ball_control', 'movement', 'sprint_speed', 'agility', 'reactions', 'balance', 
                            'power', 'jumping', 'aggression', 'positioning', 'vision', 'penalties', 
                            'composure', 'defending', 'goalkeeping', 'gk_diving', 'gk_handling', 'gk_kicking', 
                            'gk_positioning', 'gk_reflexes', 'total_stats', 'ir', 'hits', 'rwb']]
Y_mid = mid_fifa_train_final['ova']

In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(X_mid , Y_mid , test_size=0.2, random_state=42)

In [49]:
X_mid = sm.add_constant(X_mid)
model = sm.OLS(Y_mid,X_mid).fit()

In [50]:
lm_mid = linear_model.LinearRegression()
model = lm_mid.fit(X_train,Y_train)

lm_mid.score(X_train,Y_train)

0.9441306953369405

In [51]:
predictions_test_mid = lm_mid.predict(X_test)
predictions_train_mid = lm_mid.predict(X_train)

print("R2 value is = ",round(r2_score(Y_train, predictions_train_mid),2))
print("The intercept of the model is = ",lm_mid.intercept_)

mse_mid = mean_squared_error(Y_test, predictions_test_mid)
mae_mid = mean_absolute_error(Y_test, predictions_test_mid)
print("The mse of the model is = ",mse_mid)
print("The root mse of the model is = ",np.sqrt(mse_mid))
print("The mean absolute error of the model is = ",mae_mid)

R2 value is =  0.94
The intercept of the model is =  18.5071931542236
The mse of the model is =  2.6909725511045566
The root mse of the model is =  1.6404184073292267
The mean absolute error of the model is =  1.276363267014204


In [52]:
Y_predicted_mid = pd.concat([Y_test.reset_index(), pd.DataFrame(predictions_test_mid)], axis=1)
Y_predicted_mid.rename(columns = {0:'test_mid'}, inplace = True)
analysis_mid = Y_predicted_mid.sort_values(by = 'ova', ascending = False)
analysis_mid['Diff'] = (analysis_mid['ova'] - analysis_mid['test_mid']).round()
analysis_mid.sort_values(by = 'ova', ascending = False)
analysis_mid['test_mid']=analysis_mid['test_mid'].round()
analysis_mid

,index,ova,test_mid,Diff
909,2920,89,85.0,4.0
882,5809,85,86.0,-1.0
827,1970,85,85.0,0.0
787,7459,85,87.0,-2.0
42,2391,84,81.0,3.0
...,...,...,...,...
130,12051,48,52.0,-4.0
271,13573,48,48.0,-0.0
400,13051,48,50.0,-2.0
73,13325,48,53.0,-5.0


In [53]:
mid_fifa_test['hits'] = mid_fifa_test['hits'].fillna(mid_fifa_test['hits'].mean())

In [54]:
X_mid_test = mid_fifa_test[['age', 'height', 'growth', 'value', 'attacking', 'crossing', 'finishing', 
                            'heading_accuracy', 'volleys', 'skill', 'curve', 'fk_accuracy', 'long_passing', 
                            'ball_control', 'movement', 'sprint_speed', 'agility', 'reactions', 'balance', 
                            'power', 'jumping', 'aggression', 'positioning', 'vision', 'penalties', 
                            'composure', 'defending', 'goalkeeping', 'gk_diving', 'gk_handling', 'gk_kicking', 
                            'gk_positioning', 'gk_reflexes', 'total_stats', 'ir', 'hits', 'rwb']]

predictions_test_mid = lm_mid.predict(X_mid_test)
mid_fifa_test['predictions']=predictions_test_mid.round()
final_mid = mid_fifa_test[['name','value', 'predictions']].sort_values(by='predictions', ascending = False)

# Attacking start

In [55]:
att_fifa_train_final = att_fifa_train.drop(['ova','bov','id','pot','name','pos_categ','sm'], axis=1)

In [56]:
att_fifa_train_final['hits'] = att_fifa_train_final['hits'].fillna(att_fifa_train_final['hits'].mean())

In [57]:
xcols = []
ycols = []
correlation = []
max_abs_correlation = 0.9
for x in range(len(att_fifa_train_final.columns)):
    for y in range(len(att_fifa_train_final.corr())):
        if x != y:
            xcol = att_fifa_train_final.columns[x]
            ycol = att_fifa_train_final.columns[y]
            if xcol in att_fifa_train_final.columns and ycol in att_fifa_train_final.columns:
                r = att_fifa_train_final[xcol].corr(att_fifa_train_final[ycol])
                correlation.append(r)
                xcols.append(xcol)
                ycols.append(ycol)
dropped_cols = set()
counter = 0
for i in range(len(correlation)):
    if correlation[i] > abs(max_abs_correlation):
        xcol = xcols[i]
        ycol = ycols[i]
        if xcol in att_fifa_train_final.columns and ycol in att_fifa_train_final.columns:
            counter += 1
            # drop one of the columns with high correlation
            if xcol not in dropped_cols and ycol not in dropped_cols:
                if att_fifa_train_final[xcol].std() > att_fifa_train_final[ycol].std():
                    att_fifa_train_final = att_fifa_train_final.drop([ycol], axis=1)
                    dropped_cols.add(ycol)
                else:
                    att_fifa_train_final = att_fifa_train_final.drop([xcol], axis=1)
                    dropped_cols.add(xcol)
    else:
        pass

In [58]:
att_fifa_train_final = pd.concat([att_fifa_train_final, att_fifa_train['ova']], axis=1)

In [59]:
drop_col_att = ['height','wage','movement','agility','balance','power','jumping','stamina','strength','total_stats',
             'phy','rdm','skill','curve','age','fk_accuracy','gk','long_passing','ir']

att_fifa_train_final = att_fifa_train_final.drop(drop_col_att, axis=1)

In [60]:
X_att = att_fifa_train_final[['weight', 'growth', 'value', 'attacking', 'crossing', 'finishing', 'heading_accuracy',
                            'volleys', 'dribbling', 'ball_control', 'acceleration', 'sprint_speed', 'reactions',
                            'shot_power', 'long_shots', 'mentality', 'aggression', 'interceptions', 'positioning', 
                            'vision', 'penalties', 'composure', 'defending', 'marking', 'sliding_tackle', 
                            'goalkeeping', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 
                            'gk_reflexes', 'w/f', 'hits']]
Y_att = att_fifa_train_final['ova']

In [61]:
X_train, X_test, Y_train, Y_test = train_test_split(X_att , Y_att , test_size=0.2, random_state=42)

In [62]:
X_att = sm.add_constant(X_att)
model = sm.OLS(Y_att,X_att).fit()

In [63]:
lm_att = linear_model.LinearRegression()
model = lm_att.fit(X_train,Y_train)

lm_att.score(X_train,Y_train)

0.9743960847824269

In [64]:
predictions_test_att = lm_att.predict(X_test)
predictions_train_att = lm_att.predict(X_train)

print("R2 value is = ",round(r2_score(Y_train, predictions_train_att),2))
print("The intercept of the model is = ",lm_att.intercept_)

mse_att = mean_squared_error(Y_test, predictions_test_att)
mae_att = mean_absolute_error(Y_test, predictions_test_att)
print("The mse of the model is = ",mse_att)
print("The root mse of the model is = ",np.sqrt(mse_att))
print("The mean absolute error of the model is = ",mae_att)

R2 value is =  0.97
The intercept of the model is =  3.694916752249746
The mse of the model is =  1.100024002734915
The root mse of the model is =  1.04882029096262
The mean absolute error of the model is =  0.8147603425500626


In [65]:
Y_predicted_att = pd.concat([Y_test.reset_index(), pd.DataFrame(predictions_test_att)], axis=1)
Y_predicted_att.rename(columns = {0:'test_att'}, inplace = True)
analysis_att = Y_predicted_att.sort_values(by = 'ova', ascending = False)
analysis_att['Diff'] = (analysis_att['ova'] - analysis_att['test_att']).round()
analysis_att.sort_values(by = 'ova', ascending = False)
analysis_att['test_att']=analysis_att['test_att'].round()
analysis_att

,index,ova,test_att,Diff
480,111,92,93.0,-1.0
200,2138,91,92.0,-1.0
464,2679,85,84.0,1.0
49,3593,83,82.0,1.0
356,3059,82,83.0,-1.0
...,...,...,...,...
435,11616,51,53.0,-2.0
34,9963,51,50.0,1.0
387,4889,50,51.0,-1.0
403,11449,50,51.0,-1.0


In [66]:
att_fifa_test['hits'] = att_fifa_test['hits'].fillna(att_fifa_test['hits'].mean())

In [67]:
X_att_test = att_fifa_test[['weight', 'growth', 'value', 'attacking', 'crossing', 'finishing', 'heading_accuracy',
                            'volleys', 'dribbling', 'ball_control', 'acceleration', 'sprint_speed', 'reactions',
                            'shot_power', 'long_shots', 'mentality', 'aggression', 'interceptions', 'positioning', 
                            'vision', 'penalties', 'composure', 'defending', 'marking', 'sliding_tackle', 
                            'goalkeeping', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 
                            'gk_reflexes', 'w/f', 'hits']]

predictions_test_att = lm_att.predict(X_att_test)
att_fifa_test['predictions']=predictions_test_att.round()
final_att = att_fifa_test[['name','value', 'predictions']].sort_values(by='predictions', ascending = False)

# FINAL RESULT: BUDGET 100M & 3 BEST PLAYERS PER OVA

In [68]:
final_overall = pd.concat([final_att,final_mid,final_def,final_gk],axis=0)
final_overall12 = final_overall.sort_values(by='predictions', ascending=False).head(12)
final_overall12

,name,value,predictions
158,K. Mbappé,105500000.0,94.0
494,V. van Dijk,75500000.0,93.0
1385,P. Dybala,71000000.0,92.0
1912,S. Mané,78000000.0,92.0
992,M. Neuer,29000000.0,89.0
2731,F. de Jong,50500000.0,89.0
726,L. Modrić,24500000.0,89.0
1432,A. Robertson,53000000.0,88.0
1836,M. Hummels,33000000.0,88.0
2521,S. Handanovič,16000000.0,88.0


In [69]:
import itertools

budget_limit = 100000000


combinations = []
for r in range(1, len(final_overall12) + 1):
    combinations.extend(itertools.combinations(final_overall12.iterrows(), r))

best_ova = 0
best_combination = None

# Iterate through all possible combinations to find the best one with exactly 3 players

for combination in combinations:
    total_cost = 0
    total_ova = 0
    selected_players = []

    for player in combination:
        index, row = player
        if total_cost + row['value'] <= budget_limit:
            selected_players.append(row)
            total_cost += row['value']
            total_ova += row['predictions']
        
        if len(selected_players) == 3:
            break  # Limit the selection to 3 players

    if len(selected_players) == 3 and total_ova > best_ova:
        best_ova = total_ova
        best_combination = selected_players

# Create a new DataFrame with the best combination of 3 players

best_players_result = pd.DataFrame(best_combination)

best_players_result

,name,value,predictions
992,M. Neuer,29000000.0,89.0
2731,F. de Jong,50500000.0,89.0
2521,S. Handanovič,16000000.0,88.0


In [70]:
liquidity = budget_limit -sum (best_players_result['value'])
liquidity

4500000.0